In [61]:
from scipy.linalg import null_space
import numpy as np
from pwapprx import PWApprx
import time
import matplotlib.pyplot as plt
import matplotlib as mpl
import random
import copy
from numba import jit

In [2]:
np.set_printoptions(linewidth=np.inf)

In [13]:
def logistic_iterative_lyapunov(r = 2.5, rb = None, n_warmups = 2000, n_iter = 400, x_0 = .5):
    
    def logistic_derivative(x, r, epsilon = 0):
        return np.add(np.abs(np.multiply(r, (np.subtract(1, (np.multiply(2, x.T))))).T), epsilon)
    
    x = x_0
    lyapunov_ex = 0
    
    if rb != None:
        ra = r
        for i in range(n_warmups):
            if i % 2 == 0:
                x = np.multiply(x, ra) * np.subtract(1,x)
            else:
                x = np.multiply(x, rb) * np.subtract(1,x)

        for i in range(n_iter):

            if i % 2 == 0:
                x = np.multiply(x, ra) * np.subtract(1,x)
                lyapunov_ex = np.add(lyapunov_ex , np.log(logistic_derivative(x, ra)))
            else:
                x = np.multiply(x, rb) * np.subtract(1,x)
                lyapunov_ex = np.add(lyapunov_ex , np.log(logistic_derivative(x, rb)))
    
    else:
        # when n_points is zero it signifies wanting to use the logistic apprx
        
        for i in range(n_warmups):
            x = np.multiply(x, r) * np.subtract(1,x) 

        for i in range(n_iter):
            lyapunov_ex = np.add(lyapunov_ex, np.log(logistic_derivative(x, r)))
            x = np.multiply(x, r) * np.subtract(1,x) 

    lyapunov_ex = np.divide(lyapunov_ex, n_iter)
    
    if rb != None:     
        print("logistic AB:\t" + str(lyapunov_ex))
    else:
        print("logistic:\t" + str(lyapunov_ex))     
    

In [28]:
def linear_iterative_lyapunov(r = 2.5, rb = None, n_warmups = 2000, n_iter = 400, x_0 = .5, n_points = 5):
    
    def linear_derivative(x, function):
        return np.abs(function.slope(x))
    
    x_values = np.round(np.linspace(0, 1, n_points+2), 3) # values of x on the curve that will be used to create the approximation
    
    x = x_0
    lyapunov_ex = 0
    
    
    func_a  = PWApprx(x_values, r)
    
    if rb != None:
        ra = r
        func_b  = PWApprx(x_values, rb)
        
        for i in range(n_warmups):
            if i % 2 == 0:
                x = func_a.compute(x)
            else:
                x = func_b.compute(x)

        for i in range(n_iter):

            if i % 2 == 0:
                x = func_a.compute(x)
                lyapunov_ex = np.add(lyapunov_ex , np.log(linear_derivative(x, func_a)))
            else:
                x = func_b.compute(x)
                lyapunov_ex = np.add(lyapunov_ex , np.log(linear_derivative(x, func_a)))
    
    else:
        x_values = np.round(np.linspace(0, 1, n_points+2), 3)
        func = PWApprx(x_values, r)

        for i in range(n_warmups):
            x = func_a.compute(x)

        for i in range(n_iter):
            x = func_a.compute(x)
            lyapunov_ex = np.add(lyapunov_ex, np.log(linear_derivative(x, func)))
            

    lyapunov_ex = np.divide(lyapunov_ex, n_iter)
    
    if rb != None:     
        print("linear AB:\t" + str(lyapunov_ex))
    else:
        print("linear:  \t" + str(lyapunov_ex))     
    

In [92]:
# returns the weight (the proportion) of x values of a function mapped by y values of another fuction   
@jit
def calc_weight(x_lb, x_ub, y_lb, y_ub):        
    if x_lb > y_ub or y_lb > x_ub:
        return 0
    else:
        overlap_lb = x_lb if x_lb > y_lb else y_lb
        overlap_ub = x_ub if x_ub < y_ub else y_ub
        return (overlap_ub - overlap_lb) /(y_ub - y_lb)

def make_AB_matrix(x_values, func_a, func_b, p = None):
    
    n_equations = len(x_values)-1
    
    zeros = np.zeros((n_equations,n_equations))
    for z_i, z in enumerate(zeros):
        z[z_i] = -1
        
    # Build the matrix one square at a time
    
    y_values_b = func_b.get_y_values()
    top_right_square = []

    for i in range(len(x_values)):
        if i != 0:      
            w_i = []
            for j in range(len(x_values)):
                if j != 0:  
                    # Top Right Square
                    weight = calc_weight(x_values[i-1],x_values[i], min(y_values_b[j-1],y_values_b[j]),max(y_values_b[j-1],y_values_b[j]))
                    w_i.append(weight)

            top_right_square.append(w_i)

    # needed to compute lyapunov exponent
    slopes_b = func_b.get_slope_list()

    top = np.concatenate((zeros, top_right_square), axis = 1)

    y_values_a = func_a.get_y_values()
    bottom_left_square = []

    for i in range(len(x_values)):
        if i != 0:      
            w_i = []
            for j in range(len(x_values)):
                if j != 0:  
                    # Bottom Left Square
                    weight = calc_weight(x_values[i-1],x_values[i], min(y_values_a[j-1],y_values_a[j]),max(y_values_a[j-1],y_values_a[j]))
                    w_i.append(weight) 
            bottom_left_square.append(w_i)

    bottom = np.concatenate((bottom_left_square, zeros), axis = 1)

    matrix = np.concatenate((top, bottom))

    if p != None:
        matrix[-1] = p
    
    return matrix

def calc_portion(x_lb, x_ub, total_x_interval):
    p = x_ub - x_lb / total_x_interval
    return p
    
def calc_equal_portion(n, total_x_interval):
    p = (1/n) * total_x_interval
    return p 


In [88]:
def invariant_lyapunov(r = 2.5, n_points = 5):
    
    x_values = np.round(np.linspace(0, 1, n_points+2), 3) # values of x on the curve that will be used to create the approximation
    
    # create functions for linear approximation
    func = PWApprx(x_values, r)
    y_values = func.get_y_values()
    matrix = []
    p = []
    
    total_x_interval = np.amax(x_values) - np.amin(x_values)
    
    for i in range(len(x_values)):
        if i != 0:
            w_i = []
            for j in range(len(x_values)):
                if j != 0:
                    weight = calc_weight(x_values[i-1],x_values[i], min(y_values[j-1],y_values[j]),max(y_values[j-1],y_values[j]))
                    if i == j:
                        weight = weight - 1
                    w_i.append(weight)    
                    
            
            matrix.append(w_i)

#             p.append(calc_portion(x_values[i-1],x_values[i], total_x_interval))    
            p.append(calc_equal_portion(len(x_values)-1, total_x_interval))

    matrix = np.asarray(matrix)   
    matrix[-1] = p
    
    ans = np.zeros(len(matrix))
    ans[-1] = 1
    
#     print(matrix) 
#     print(ans)
    
    solution = np.linalg.solve(matrix, ans)
 
    slopes = func.get_slope_list()
    
    lyap = np.sum(p * solution * np.log(np.abs(slopes)))
    
    print("invariant:\t" + str(lyap))
    

In [227]:
def invariant_lyapunov_AB(ra = 2.5, rb = 2.5, n_points = 5):
  
    x_values = np.round(np.linspace(0, 1, n_points+2), 3) # values of x on the curve that will be used to create the approximation
    
    n_equations = n_points+1
    
    total_x_interval = np.amax(x_values) - np.amin(x_values)
    p = [(calc_equal_portion(n_equations, total_x_interval)/2) for i in range(n_equations * 2)]
  
    # create functions for linear approximation
    
    func_a = PWApprx(x_values, ra)
    func_b = PWApprx(x_values, rb)
    
    matrix = make_AB_matrix(x_values, func_a, func_b, p)
    
    ans = np.zeros(len(matrix))
    ans[-1] = 1
    
#     print(matrix)
    
    slopes_a = func_a.get_slope_list()
    slopes_b = func_b.get_slope_list()
    
    slopes = slopes_a + slopes_b
    
    try:
        solution = np.linalg.solve(matrix, ans)
        
        lyapunov_ex = np.sum(p * solution * np.log(np.abs(slopes)))
    except:
        lyapunov_ex = 'Singularity'
    
    print("invariant AB:\t" + str(lyapunov_ex))
    
#     for i in range(n_equations*2):
#         if np.abs(solution[i]) > 1e-15:
#             print(i,": [", solution[i], "]") #, p[i]*solution[i]*np.log(abs(slopes[i])))
    
    


In [230]:
def nullspace_lyapunov_AB(ra, rb, n_points = 5):
    
    x_values = np.round(np.linspace(0, 1, n_points+2), 3)
    
    func_a = PWApprx(x_values, ra)
    func_b = PWApprx(x_values, rb)
    
    n_equations = n_points+1
    total_x_interval = np.amax(x_values) - np.amin(x_values)
    
    matrix = make_AB_matrix(x_values, func_a, func_b)
    
    slopes_a = func_a.get_slope_list()
    slopes_b = func_b.get_slope_list()
    
    slopes = slopes_a + slopes_b
    
    ns = null_space(matrix)
#     print("-"*50)
    #print(ns * np.sign(ns[0,0]))
#     print(ns)

#     print(f"mul1 is {mul1}, and mul2 is {mul2}")
    ns_transpose = ns.T
    
    for wi in range(len(squares)):
           
        # Sum Weighted method
        # sum of squares = 1
        # sum != 1
        # sum/sum = 1
        sum_weight = np.sum(ns_transpose[wi])
#         print(sum_weight)
        
        solution = np.divide(ns_transpose[wi], sum_weight)
        # now the sum of s will equal 1
        
        lyapunov_ex = np.dot(solution, np.log(np.abs(slopes)))
        print("nullspace AB:\t" + str(lyapunov_ex))
    

In [235]:
r = 3.5
rb = r
n_points = 251

logistic_iterative_lyapunov(r)

# start = time.time()
logistic_iterative_lyapunov(r, rb)
# print(f"this took {time.time() - start}")

linear_iterative_lyapunov(r, n_points = n_points)
linear_iterative_lyapunov(r, rb, n_points = n_points)

# start = time.time()
invariant_lyapunov(r, n_points = n_points)
# print(f"this took {time.time() - start}")

# start = time.time()
invariant_lyapunov_AB(r, rb, n_points = n_points)
# print(f"this took {time.time() - start}")

nullspace_lyapunov_AB(r, rb, n_points = n_points)


logistic:	-0.8725073457794719
logistic AB:	-0.8725073457794718
linear:  	-0.6687933018707167
linear AB:	-0.6687933018707167
invariant:	-0.44777478822629374
invariant AB:	-0.44777478822629374
nullspace AB:	-0.4477747882262952
nullspace AB:	-0.44777478822629424


In [1]:
def solver_race(ra, rb, n_points = 5):
    
    def calc_portion(x_lb, x_ub, total_x_interval):
        p = x_ub - x_lb / total_x_interval
        return p
    
    def calc_equal_portion(n, total_x_interval):
        p = (1/n) * total_x_interval
        return p
        
    # returns the weight (the proportion) of x values of a function mapped by y values of another fuction   
    def calc_weight(x_lb, x_ub, y_lb, y_ub):
        overlap_lb = max(x_lb,y_lb)
        overlap_ub = min(x_ub,y_ub)
        
        if overlap_lb > overlap_ub:
            return 0
        else:
            w = (overlap_ub - overlap_lb) /(y_ub - y_lb)
        return w    
    
    x_values = np.round(np.linspace(0, 1, n_points+2), 3) # values of x on the curve that will be used to create the approximation
    
    # create functions for linear approximation
    
    func_a = PWApprx(x_values, ra)
    func_b = PWApprx(x_values, rb)
    y_values_a = func_a.get_y_values()
    y_values_b = func_b.get_y_values()
    matrix = []
    p = []
    
    total_x_interval = np.amax(x_values) - np.amin(x_values)
    
    for i in range(len(x_values)):
        if i != 0:        
            w_i = []
            w_z = []
            for j in range(len(y_values_a)):
                if j != 0:
#                     weight = calc_weight(x_values[i-1],x_values[i], min(y_values_a[j-1],y_values_a[j]),max(y_values_a[j-1],y_values_a[j]))
                    weight = 0
                    if i == j:
                        weight = weight - 1
                    w_z.append(weight)
                    
                    weight = calc_weight(x_values[i-1],x_values[i], min(y_values_b[j-1],y_values_b[j]),max(y_values_b[j-1],y_values_b[j]))
#                     if i == j:
#                         weight = weight - 1
                    w_i.append(weight)    
                
            matrix.append((w_z + w_i))
            
    for i in range(len(x_values)):
        if i != 0:      
            w_i = [] 
            w_z = []
            for j in range(len(y_values_b)):
                if j != 0: 
                    weight = calc_weight(x_values[i-1],x_values[i], min(y_values_a[j-1],y_values_a[j]),max(y_values_a[j-1],y_values_a[j]))
#                     if i == j:
#                         weight = weight - 1
                    w_i.append(weight)
    
                    weight = 0
                    if i == j:
                        weight = weight - 1
                    w_z.append(weight)   
        
            matrix.append((w_i + w_z))

            
            # Proportions
            p.append(calc_equal_portion(len(x_values)-1, total_x_interval)/2)
            p.append(calc_equal_portion(len(x_values)-1, total_x_interval)/2)
                 

    matrix = np.asarray(matrix)
    
#     for col in matrix.T:
#         print(sum(col))  
    
#     print(matrix) 
    
    matrix[-1] = p
    
    ans = np.zeros(len(matrix))
    ans[-1] = 1
    
    slopes_a = func_a.get_slope_list()
    slopes_b = func_b.get_slope_list()
    
    slopes = slopes_a + slopes_b
    
#     print(matrix) 
#     print(ans)
    
    trials = 10000
    
#     start = time.time()
#     for i in range(trials):
#         solution = np.linalg.solve(matrix, ans)
#     end = time.time()
#     ex = np.sum(p * solution * np.log(np.abs(slopes)))
#     duration = end - start
#     print('np solver: {:3.5}, completed in {:5.5} seconds'.format(ex, duration))
    
    start = time.time()
    for i in range(trials):   
        csrow = csr_matrix(matrix)
    end = time.time()
    duration = end - start
    print('csr converstion completed in {:5.5} seconds'.format(duration))
    
    start = time.time()
    for i in range(trials):
        solution = spsolve(csrow, ans)
    end = time.time()
    ex = np.sum(p * solution * np.log(np.abs(slopes)))
    duration = end - start
    print('scipy csr solver: {:1.5}, completed in {:5.5} seconds'.format(ex, duration))
    
    start = time.time()
    for i in range(trials):   
        cscol = csc_matrix(matrix)
    end = time.time()
    duration = end - start
    print('csc converstion completed in {:5.5} seconds'.format(duration))
    
    start = time.time()
    for i in range(trials):
        solution = spsolve(cscol, ans)
    end = time.time()
    ex = np.sum(p * solution * np.log(np.abs(slopes)))
    duration = end - start
    print('scipy csc solver: {:1.5}, completed in {:5.5} seconds'.format(ex, duration))
    

In [237]:
solver_race(3.1, 3.1, n_points = 11)
solver_race(3.1022044088176353, 3.090180360721443, n_points = 251)

NameError: name 'csr_matrix' is not defined